In [1]:
import csv
import re
import timeit
import datetime
from datetime import timedelta
import psycopg2
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [2]:
smtp_server = "smtp.gmail.com"
sender_email = "rise.python.automail@gmail.com"  # Enter your address
receiver_email = "chongdi0505@gmail.com"  # Enter receiver address
password = "rise654321"

In [3]:
output_header = ["Ticket",
                 "Date",
                 "Time",
                 "Header",
                 "County",
                 "Rating",
                 "Longitude",
                 "Latitude",
                 "St_number",
                 "St_name",
                 "Neighborhood",
                 "City",
                 "State",
                 "Zip",
                 "Excav_company",
                 "Inter1",
                 "Inter2"]

In [4]:
def email_notification(subject, message):
    
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    # Attach the message to the MIMEMultipart object
    msg.attach(MIMEText(message, 'plain'))

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(sender_email, password)
    text = msg.as_string() # You now need to convert the MIMEMultipart object to a string to send
    server.sendmail(sender_email, receiver_email, text)
    server.quit()

In [5]:
def session_start_msg(input_path, output_path, input_file, entries):
    
    start_time = datetime.datetime.now()
    processing_duration_sec = entries * 7
    finish_time = start_time + timedelta(seconds = processing_duration_sec)
    
    subject = "Geocoding Session Started"
    
    start_message = """\
    Job started at %d:%d:%d %d-%d-%d
    
    Input location: %s
    Output location: %s
    Input file: %s
    
    Total input entries: %d
    Estimated processing time: %d hours %d minutes(7 seconds per entry)
    Anticipated finishing time: %d:%d:%d %d-%d-%d
    """ % (
        start_time.time().hour,
        start_time.time().minute,
        start_time.time().second,
        start_time.date().month,
        start_time.date().day,
        start_time.date().year,
        input_path,
        output_path,
        input_file,
        entries,
        processing_duration_sec // 3600,
        processing_duration_sec % 3600 // 60,
        finish_time.time().hour,
        finish_time.time().minute,
        finish_time.time().second,
        finish_time.date().month,
        finish_time.date().day,
        finish_time.date().year,
            )
    return(subject, start_message)
    
#sbj, messageg = session_start_msg('./', './', 'ticketSearchResults-2015-02-09-20_48_16.txt', 13600)

In [6]:
def session_finish_msg(input_path, output_path, input_file, processed_indices, ungeocoded_indices, total_lines):
    
    stop_time = datetime.datetime.now()
    ungeocoded_rate = ungeocoded_indices / (processed_indices + 1) * 100
    
    subject = "Geocoding Session Complete"

    start_message = """\
    Job finished at %d:%d:%d %d-%d-%d

    Input location: %s
    Output location: %s
    Input file: %s

    Total processed entries: %d
    Ungeocoded entries: %d
    Ungeocoded rate: %.2f%%
    """ % (
        stop_time.time().hour,
        stop_time.time().minute,
        stop_time.time().second,
        stop_time.date().month,
        stop_time.date().day,
        stop_time.date().year,
        input_path,
        output_path,
        input_file,
        processed_indices + 1,
        ungeocoded_indices,
        ungeocoded_rate
            )
    return(subject, start_message)

In [7]:
def geocoder(input_path, output_path, input_file):

    # preparing for geocoding
    try:
        conn = psycopg2.connect("dbname='geocoder' user='postgres' host='localhost' password='postgres'")
        print("Database connected!", end='\r')
        cur = conn.cursor()
    except:
        print("Python is not able to connect to the database!", end='\r')
    
    f_in = open(input_path + input_file, 'r')
    f_out = open(output_path + input_file.replace('.txt', '.csv'), 'w')
    f_bad = open(output_path + input_file.replace('emergency', 'badlines'), 'w')
    f_log = open(output_path + input_file.replace('.txt', '.log'), 'w')
    
    write_header = csv.writer(f_out, lineterminator='\n')
    write_header.writerow(output_header)
    
    lines = f_in.readlines()
    total_entry = len(lines)
    
    # send email notification when session started
    start_sbj, start_msg = session_start_msg(input_path, output_path, input_file, total_entry)
    email_notification(start_sbj, start_msg)
    
    # geocoding
    
    i = 0 # index for all processed entries
    n = 0 # index for all ungeocoded entries
    
    for line in lines:
        if not re.search(r'Ticket', line):
            rec = line.split('~')
            address = (rec[6], # street number
                       rec[7].replace("'", "''"), # street name
                       rec[4]) # city name

            expression = """SELECT g.rating, ST_X(g.geomout), ST_Y(g.geomout), (addy).address, (addy).streetname, (addy).location, (addy).stateabbrev, (addy).zip FROM geocode('%s %s, %s, NJ', 1) AS g;""" % address

            cur.execute(expression)

            try:

                rows = [str(x) for x in cur.fetchall()[0]]

                f_out.write("%s,%s,%s,%s,%s," % (
                    rec[0], # ticket
                    rec[1].split(' ')[0], # date
                    rec[1].split(' ')[1], # time
                    rec[2], # header
                    rec[3]  # county
                ))

                f_out.write("%s,%s,%s,%s,%s,%s,%s,%s," % (
                    rows[0], # rating int
                    rows[1], # longitude float.13
                    rows[2], # latitude float.13
                    rows[3], # street number int
                    rows[4], # street name string
                    rows[5], # neighborhood string
                    rows[6], # state
                    rows[7]  # zip
                ))

                f_out.write("%s,%s,%s\n" % (
                    rec[5], # excav_comp
                    rec[8], # inter1
                    rec[9].strip(), # inter2
                ))

                print("Job index %d status: complete!" % i, end='\r')
                f_log.write("Job index %d status: complete!\n" % i)

            except:
                f_bad.write("%s\n" % line)
                n += 1
                print("\rJob index %d status: bad input line!(Total: %d)" % (i, n), end='\r')
                f_log.write("Job index %d status: bad input line!(Total: %d)\n" % (i, n))

            i += 1

    # finalize geocoding
    f_in.close()
    f_out.close()
    f_bad.close()
    f_log.close()
    print("Geocoding complete!", end='\r')
    print("Input/Output files closed!", end='\r')
    
    if conn:
        conn.close()
        print("Database disconnected!", end='\r')
    
    # send email notification when session complete
    stop_sbj, stop_msg = session_finish_msg(input_path, output_path, input_file, i, n, total_entry)
    email_notification(stop_sbj, stop_msg)

In [24]:
import os

total_f = [x for x in os.listdir('H:geocoding/reduced_input/') if re.search(r'emergency', x)]
processed_f = [x.strip() for x in open('H:geocoding/reduced_input/processed.txt').readlines()]

In [25]:
unprocessed_f = set(total_f) - set(processed_f)

In [26]:
import numpy as np
from numpy import random

In [27]:
unprocessed_f

{'emergency_2015-02-09-21_44_38.txt',
 'emergency_2015-02-09-21_44_59.txt',
 'emergency_2015-02-09-21_45_12.txt',
 'emergency_2015-02-09-21_45_27.txt',
 'emergency_2015-02-09-21_45_44.txt',
 'emergency_2015-02-09-21_46_01.txt',
 'emergency_2015-02-09-21_46_17.txt',
 'emergency_2015-02-09-21_46_37.txt',
 'emergency_2015-02-09-21_46_56.txt',
 'emergency_2015-02-09-21_47_07.txt',
 'emergency_2015-02-09-21_47_23.txt',
 'emergency_2015-02-09-21_47_41.txt',
 'emergency_2015-02-09-21_47_57.txt',
 'emergency_2015-02-09-21_49_11.txt',
 'emergency_2015-02-09-21_49_43.txt',
 'emergency_2015-02-09-21_50_03.txt',
 'emergency_2015-02-09-21_50_26.txt',
 'emergency_2015-02-09-21_50_47.txt',
 'emergency_2015-02-09-21_51_03.txt',
 'emergency_2015-02-09-21_51_30.txt',
 'emergency_2015-02-09-21_51_43.txt',
 'emergency_2015-02-09-21_52_05.txt',
 'emergency_2015-02-09-21_52_19.txt',
 'emergency_2015-02-09-21_52_44.txt',
 'emergency_2015-02-09-21_52_58.txt',
 'emergency_2015-02-09-21_53_25.txt',
 'emergency_

In [28]:
len(unprocessed_f)

30

In [ ]:
lst_next = ['emergency_2015-02-09-21_44_38.txt',
 'emergency_2015-02-09-21_44_59.txt',
 'emergency_2015-02-09-21_45_12.txt',
 'emergency_2015-02-09-21_45_27.txt',
 'emergency_2015-02-09-21_45_44.txt',
 'emergency_2015-02-09-21_46_01.txt',
 'emergency_2015-02-09-21_46_17.txt',
 'emergency_2015-02-09-21_46_37.txt',
 'emergency_2015-02-09-21_46_56.txt',
 'emergency_2015-02-09-21_47_07.txt']

for job in lst_next:
    geocoder("H:/geocoding/reduced_input/", "H:/geocoding/reduced_input/geocode_out/", job)
    processed_rec = open('H:geocoding/reduced_input/processed.txt', 'a')
    processed_rec.write('\n%s' % job)
    processed_rec.close()